In [2]:
import torch
from torchvision import datasets
from torch.utils.data import Dataset
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models as torchvision_models
import os
import shutil
from pathlib import Path
from torch.cuda.amp import autocast
from tqdm import tqdm
from kmeans_pytorch import kmeans
import wandb

import torchvision

import sys
sys.path.append("../")
from src.utils import ResNet18, transform_train, transform_test

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torchvision_archs = sorted(name for name in torchvision_models.__dict__
                           if name.islower() and not name.startswith("__")
                           and callable(torchvision_models.__dict__[name]))

print(device)

cuda


In [7]:
class ReturnIndexDataset(datasets.ImageFolder):
    def __getitem__(self, idx):
        img, lab = super(ReturnIndexDataset, self).__getitem__(idx)
        # path = super(ReturnIndexDataset, self).samples[idx]
        return idx, img, lab, idx

In [8]:
def find_class_means(X, labels, num_clusters):
    dim = X[0].shape[0]
    labels_sum = {i: torch.zeros(dim) for i in range(num_clusters)}
    labels_count = {i: 0 for i in range(num_clusters)}
    for i in range(len(X)):
        tensor = X[i]
        label = int(labels[i].item())
        labels_sum[label] += tensor
        labels_count[label] += 1
    labels_mean_tensor = torch.zeros((num_clusters, dim))
    for i in range(num_clusters):
        labels_mean_tensor[i] = labels_sum[i] / labels_count[i]
    return labels, labels_mean_tensor


In [9]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [10]:
def find_desired_samples(reps, indices, labels, base_dataset, target_dataset, cluster_centers, cluster_ids_x, quantile):
    res_values = []
    res_indices = []
    res_class_labels = []
    res_cluster_labels = []

    batch_size = 16
    num_clusters = len(cluster_centers)
    reps_dataset = CustomDataset(reps.detach())
    reps_dataloader = DataLoader(reps_dataset, batch_size=batch_size, shuffle=False)

    indices = torch.squeeze(indices)
    labels = torch.squeeze(labels)
    cluster_ids_x = torch.squeeze(cluster_ids_x)
    cluster_centers = cluster_centers.to(device)

    # calculate norm
    i = 0
    for tensor in tqdm(reps_dataloader, desc='Calculating norms'):
        tensor = tensor.to(device)
        norm_tensor = torch.linalg.norm(tensor.unsqueeze(dim=1) - cluster_centers.unsqueeze(dim=0), dim=2).detach()
        norm_tensor, norm_tensor_indecies = torch.sort(norm_tensor, dim=1)
        res_values += (norm_tensor[:, 0] - norm_tensor[:, 1] - norm_tensor[:, 2]).tolist()
        res_indices += (indices[batch_size * i: (i + 1) * batch_size]).tolist()
        res_class_labels += (labels[batch_size * i: (i + 1) * batch_size]).tolist()
        res_cluster_labels += norm_tensor_indecies[:, 0].tolist()
        i += 1

    # reordering samples and finding quantiles baesd on each class
    cluster_scores = {k: [res_values[i] for i in range(len(res_values)) if int(res_cluster_labels[i]) == k] for k in
                        range(len(cluster_centers))}

    quantiles = {k: torch.quantile(torch.tensor(cluster_scores[k]), q=quantile) for k in
                    range(num_clusters) if len(cluster_scores[k]) != 0}
    score_dicts = {int(res_indices[i]): (res_values[i], int(res_class_labels[i]), int(res_cluster_labels[i])) for i
                    in
                    range(len(res_values))}
    results_based_on_class = {i: [] for i in range(len(target_dataset.classes))}

    # finding images which are in the quantile period
    for k, v in tqdm(score_dicts.items(), desc='Finding images in quntile'):
        if v[0] > quantiles[v[2]].item():
            results_based_on_class[v[1]].append(k)

    # find path of desired samples
    img_paths = {}
    for idx, img, label, ind in tqdm(target_dataset, desc='Gathering paths of desired samples'):
        image_path = target_dataset.samples[idx][0]
        if ind in results_based_on_class[label]:
            try:
                img_paths[label].append(image_path)
            except KeyError:
                img_paths[label] = [image_path]

    return img_paths

def reverse_normalization(images):
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    un_normalize = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())
    return un_normalize(images)


def save_outputs(dst_path, dataset, farthest_samples_paths):
    try:
        shutil.rmtree(dst_path)
    except FileNotFoundError:
        pass
    Path(dst_path).mkdir(parents=True, exist_ok=True)
    for cls in dataset.classes:
        Path(os.path.join(dst_path, str(cls))).mkdir(parents=True, exist_ok=True)
    for cls, paths in farthest_samples_paths.items():
        for i, path in enumerate(paths):
            shutil.copy(path, os.path.join(dst_path, dataset.classes[cls]))


def generate_representations(batch_size, model, dataloader, dataset, desc=''):
    model.eval()

    reps = torch.zeros((len(dataloader) * batch_size, 1000))
    indices = torch.zeros((len(dataloader) * batch_size, 1))
    labels = torch.zeros((len(dataloader) * batch_size, 1))
    i = 0
    for idx, tensor, label, index in tqdm(dataloader, desc=desc):
        tensor = tensor.to(device)
        with autocast(enabled=True):
            feats = model(tensor)
        reps[i * batch_size: min((i + 1) * batch_size, len(dataset))] = feats.detach().cpu()
        labels[i * batch_size: min((i + 1) * batch_size, len(dataset))] = label[:, None]
        indices[i * batch_size: min((i + 1) * batch_size, len(dataset))] = index[:, None]
        i += 1
    return reps, indices, labels

In [11]:
# pretrained resnet-18 model
model = torch.hub.load('facebookresearch/swav:main', 'resnet50', pretrained=True)
model = model.to(device)

Downloading: "https://github.com/facebookresearch/swav/zipball/main" to /home/studio-lab-user/.cache/torch/hub/main.zip
/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/deepcluster/swav_800ep_pretrain.pth.tar" to /home/studio-lab-user/.cache/torch/hub/checkpoints/swav_800ep_pretrain.pth.tar
100%|██████████| 108M/108M [00:00<00:00, 325MB/s] 


In [12]:
cifar_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)

# Create directories for each class
classes = cifar_dataset.classes
data_dir = './cifar10_data'

for cls in classes:
    os.makedirs(os.path.join(data_dir, cls), exist_ok=True)

import torchvision.transforms.functional as TF

# Move images to respective class directories
for idx, (image, label) in enumerate(cifar_dataset):
    class_dir = os.path.join(data_dir, classes[label])
    image_path = os.path.join(class_dir, f"img_{idx}.jpg")
    tensor_image = TF.to_tensor(image)  # Convert PIL image to tensor
    torchvision.utils.save_image(tensor_image, image_path)

print("CIFAR10 dataset downloaded and organized successfully.")

100%|██████████| 170498071/170498071 [00:01<00:00, 98740619.49it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
CIFAR10 dataset downloaded and organized successfully.


In [13]:
def get_data(data_path):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    dataset = ReturnIndexDataset(data_path, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=16,
        num_workers=2,
        pin_memory=True,
        drop_last=False,
        shuffle=True
    )
    return dataloader, dataset

In [15]:
dataloader, dataset = get_data("./cifar10_data")

In [16]:
reps, indices, labels = generate_representations(
    batch_size=16,
    model=model,
    dataloader=dataloader,
    dataset=dataset,
    desc='Generating representations'
)

Generating representations:   0%|          | 0/3125 [00:00<?, ?it/s]/opt/conda/envs/sagemaker-distribution/lib/python3.10/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Generating representations: 100%|██████████| 3125/3125 [04:28<00:00, 11.62it/s]


In [17]:
data_size, dims = reps.shape
num_clusters = len(dataset.classes)

cluster_ids_x, cluster_centers = kmeans(X=reps,
                                        num_clusters=num_clusters,
                                        distance='euclidean',
                                        device=device,
                                        tol=1e-5)

running k-means on cpu..


[running kmeans]: 87it [02:46,  1.92s/it, center_shift=0.000000, iteration=87, tol=0.000010] 


Quantile 50

In [18]:
farthest_samples_paths = find_desired_samples(reps=reps,
                                                indices=indices,
                                                labels=labels,
                                                base_dataset=dataset,
                                                target_dataset=dataset,
                                                cluster_centers=cluster_centers,
                                                cluster_ids_x=cluster_ids_x,
                                                quantile=0.5
                                              )

save_outputs(dst_path='./hard_samples_50/',
                 dataset=dataset,
                 farthest_samples_paths=farthest_samples_paths)

Gathering paths of desired samples: 100%|██████████| 50000/50000 [00:29<00:00, 1704.26it/s]


Quantile 10

In [19]:
farthest_samples_paths = find_desired_samples(reps=reps,
                                                indices=indices,
                                                labels=labels,
                                                base_dataset=dataset,
                                                target_dataset=dataset,
                                                cluster_centers=cluster_centers,
                                                cluster_ids_x=cluster_ids_x,
                                                quantile=0.1
                                              )

save_outputs(dst_path='./hard_samples_10/',
                 dataset=dataset,
                 farthest_samples_paths=farthest_samples_paths)

Gathering paths of desired samples: 100%|██████████| 50000/50000 [00:24<00:00, 2034.70it/s]


Quantile 20

In [20]:
farthest_samples_paths = find_desired_samples(reps=reps,
                                                indices=indices,
                                                labels=labels,
                                                base_dataset=dataset,
                                                target_dataset=dataset,
                                                cluster_centers=cluster_centers,
                                                cluster_ids_x=cluster_ids_x,
                                                quantile=0.2
                                              )

save_outputs(dst_path='./hard_samples_20/',
                 dataset=dataset,
                 farthest_samples_paths=farthest_samples_paths)

Gathering paths of desired samples: 100%|██████████| 50000/50000 [00:25<00:00, 1947.85it/s]


In [23]:
!find hard_samples_50 -mindepth 1 -type d -exec sh -c 'echo -n "{}: "; ls -1 "{}" | wc -l' \;

hard_samples_50/airplane: 2206
hard_samples_50/automobile: 2506
hard_samples_50/bird: 2289
hard_samples_50/cat: 2502
hard_samples_50/deer: 2771
hard_samples_50/dog: 2561
hard_samples_50/frog: 2561
hard_samples_50/horse: 2546
hard_samples_50/ship: 2358
hard_samples_50/truck: 2698


### Train

In [5]:
import torch
import torchvision
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

In [6]:
prune_frac = "10"
wandb.init(project='cifar10_pruning', name='pruned-neural-scaling-'+prune_frac)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_folder = "./hard_samples_"+ prune_frac

In [8]:
trainset = datasets.ImageFolder(train_folder, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Initialize the ConvNet model
net = ResNet18().to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Train the ConvNet
start_time = time.time()
for epoch in range(30):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data


        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            wandb.log({"Loss": running_loss / 200}, step=epoch * len(trainloader) + i)
            # Log the loss to wandb, so that we can visualize it
            running_loss = 0.0
            step_val = epoch * len(trainloader) + i + 1

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('[epoch:%d,  accuracy: %.3f' % (epoch + 1, accuracy))
    wandb.log({"Accuracy": accuracy}, step=step_val)

    scheduler.step()


end_time = time.time()
training_time = end_time - start_time

# Test the ConvNet on the test set
correct = 0
total = 0
top5_correct = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        _, top5_predicted = torch.topk(outputs.data, 5, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        top5_correct += (top5_predicted == labels.view(-1, 1)).sum().item()

accuracy = 100 * correct / total
top5_accuracy = 100 * top5_correct / total
wandb.log({"Final-Accuracy": accuracy, "Top-5 Accuracy": top5_accuracy, "Training Time": training_time})

Files already downloaded and verified
[1,   200] loss: 1.555
[epoch:1,  accuracy: 60.840
[2,   200] loss: 0.955
[epoch:2,  accuracy: 69.970
[3,   200] loss: 0.720
[epoch:3,  accuracy: 75.210
[4,   200] loss: 0.574
[epoch:4,  accuracy: 77.600
[5,   200] loss: 0.458
[epoch:5,  accuracy: 77.830
[6,   200] loss: 0.355
[epoch:6,  accuracy: 79.890
[7,   200] loss: 0.267
[epoch:7,  accuracy: 79.540
[8,   200] loss: 0.188
[epoch:8,  accuracy: 79.710
[9,   200] loss: 0.125
[epoch:9,  accuracy: 79.770
[10,   200] loss: 0.088
[epoch:10,  accuracy: 79.790
[11,   200] loss: 0.079
[epoch:11,  accuracy: 79.810
[12,   200] loss: 0.066
[epoch:12,  accuracy: 80.420
[13,   200] loss: 0.051
[epoch:13,  accuracy: 79.750
[14,   200] loss: 0.047
[epoch:14,  accuracy: 79.840
[15,   200] loss: 0.041
[epoch:15,  accuracy: 80.610
[16,   200] loss: 0.046
[epoch:16,  accuracy: 79.060
[17,   200] loss: 0.051
[epoch:17,  accuracy: 80.700
[18,   200] loss: 0.037
[epoch:18,  accuracy: 79.410
[19,   200] loss: 0.030
[e

Prune 20

In [9]:
prune_frac = "20"
wandb.init(project='cifar10_pruning', name='pruned-neural-scaling-'+prune_frac)
train_folder = "./hard_samples_"+ prune_frac

trainset = datasets.ImageFolder(train_folder, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Initialize the ConvNet model
net = ResNet18().to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Train the ConvNet
start_time = time.time()
for epoch in range(30):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data


        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            wandb.log({"Loss": running_loss / 200}, step=epoch * len(trainloader) + i)
            # Log the loss to wandb, so that we can visualize it
            running_loss = 0.0
            step_val = epoch * len(trainloader) + i + 1

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('[epoch:%d,  accuracy: %.3f' % (epoch + 1, accuracy))
    wandb.log({"Accuracy": accuracy}, step=step_val)

    scheduler.step()


end_time = time.time()
training_time = end_time - start_time

# Test the ConvNet on the test set
correct = 0
total = 0
top5_correct = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        _, top5_predicted = torch.topk(outputs.data, 5, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        top5_correct += (top5_predicted == labels.view(-1, 1)).sum().item()

accuracy = 100 * correct / total
top5_accuracy = 100 * top5_correct / total
wandb.log({"Final-Accuracy": accuracy, "Top-5 Accuracy": top5_accuracy, "Training Time": training_time})

Accuracy,▁▄▆▇▇██████████▇██████████████
Final-Accuracy,▁
Loss,█▅▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Top-5 Accuracy,▁
Training Time,▁
Accuracy,79.64
Final-Accuracy,79.64
Loss,0.01474
Top-5 Accuracy,98.43
Training Time,1134.00164


Files already downloaded and verified
[1,   200] loss: 1.558
[epoch:1,  accuracy: 58.700
[2,   200] loss: 1.003
[epoch:2,  accuracy: 68.870
[3,   200] loss: 0.790
[epoch:3,  accuracy: 74.590
[4,   200] loss: 0.612
[epoch:4,  accuracy: 76.840
[5,   200] loss: 0.497
[epoch:5,  accuracy: 77.390
[6,   200] loss: 0.384
[epoch:6,  accuracy: 78.550
[7,   200] loss: 0.294
[epoch:7,  accuracy: 80.140
[8,   200] loss: 0.211
[epoch:8,  accuracy: 79.520
[9,   200] loss: 0.136
[epoch:9,  accuracy: 79.120
[10,   200] loss: 0.107
[epoch:10,  accuracy: 79.520
[11,   200] loss: 0.089
[epoch:11,  accuracy: 79.580
[12,   200] loss: 0.072
[epoch:12,  accuracy: 79.210
[13,   200] loss: 0.046
[epoch:13,  accuracy: 79.530
[14,   200] loss: 0.061
[epoch:14,  accuracy: 80.170
[15,   200] loss: 0.045
[epoch:15,  accuracy: 79.740
[16,   200] loss: 0.046
[epoch:16,  accuracy: 79.440
[17,   200] loss: 0.036
[epoch:17,  accuracy: 79.370
[18,   200] loss: 0.037
[epoch:18,  accuracy: 80.110
[19,   200] loss: 0.042
[e

Prune 50

In [10]:
prune_frac = "50"
wandb.init(project='cifar10_pruning', name='pruned-neural-scaling-'+prune_frac)
train_folder = "./hard_samples_"+ prune_frac

trainset = datasets.ImageFolder(train_folder, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Initialize the ConvNet model
net = ResNet18().to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Train the ConvNet
start_time = time.time()
for epoch in range(30):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data


        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            wandb.log({"Loss": running_loss / 200}, step=epoch * len(trainloader) + i)
            # Log the loss to wandb, so that we can visualize it
            running_loss = 0.0
            step_val = epoch * len(trainloader) + i + 1

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('[epoch:%d,  accuracy: %.3f' % (epoch + 1, accuracy))
    wandb.log({"Accuracy": accuracy}, step=step_val)

    scheduler.step()


end_time = time.time()
training_time = end_time - start_time

# Test the ConvNet on the test set
correct = 0
total = 0
top5_correct = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        _, top5_predicted = torch.topk(outputs.data, 5, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        top5_correct += (top5_predicted == labels.view(-1, 1)).sum().item()

accuracy = 100 * correct / total
top5_accuracy = 100 * top5_correct / total
wandb.log({"Final-Accuracy": accuracy, "Top-5 Accuracy": top5_accuracy, "Training Time": training_time})

Accuracy,▁▄▆▇▇▇████████████████████████
Final-Accuracy,▁
Loss,█▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Top-5 Accuracy,▁
Training Time,▁
Accuracy,79.45
Final-Accuracy,79.45
Loss,0.01758
Top-5 Accuracy,98.54
Training Time,1016.97523


Files already downloaded and verified
[epoch:1,  accuracy: 53.010
[epoch:2,  accuracy: 60.620
[epoch:3,  accuracy: 67.130
[epoch:4,  accuracy: 70.040
[epoch:5,  accuracy: 72.520
[epoch:6,  accuracy: 73.810
[epoch:7,  accuracy: 74.090
[epoch:8,  accuracy: 75.330
[epoch:9,  accuracy: 74.810
[epoch:10,  accuracy: 75.570
[epoch:11,  accuracy: 75.540
[epoch:12,  accuracy: 75.870
[epoch:13,  accuracy: 74.890
[epoch:14,  accuracy: 75.610
[epoch:15,  accuracy: 76.210
[epoch:16,  accuracy: 75.050
[epoch:17,  accuracy: 76.490
[epoch:18,  accuracy: 75.780
[epoch:19,  accuracy: 76.330
[epoch:20,  accuracy: 75.430
[epoch:21,  accuracy: 75.250
[epoch:22,  accuracy: 76.040
[epoch:23,  accuracy: 75.830
[epoch:24,  accuracy: 75.370
[epoch:25,  accuracy: 75.010
[epoch:26,  accuracy: 76.160
[epoch:27,  accuracy: 73.760
[epoch:28,  accuracy: 75.870
[epoch:29,  accuracy: 76.370
[epoch:30,  accuracy: 74.580
